<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>


# Opération avancées en pandas

## Complément - niveau intermédiaire

### Introduction

Pandas supporte des opérations de manipulation des Series et DataFrame qui sont similaires dans l'esprit à ce que l'on peut faire avec une base de données et le langage SQL, mais de manière plus intuitive et expressive et beaucoup plus efficacement puisque les opérations se déroulent toutes en mémoire. 

Vous pouvez concaténer (`concat`) des DataFrame, faire des jointures (`merge`), faire des regroupements (`groupby`) ou réorganiser les indexes (`pivot`).

Nous allons dans la suite développer ces différentes techniques. 

### Concaténations avec `concat`

`concat` est utilisé pour concaténer des Series ou des DataFrames. Regardons un exemple.

In [11]:
import numpy as np
import pandas as pd

s1 = pd.Series([30, 35], index=['alice', 'bob'])
s2 = pd.Series([32, 22, 29], index=['bill', 'alice', 'jo'])
pd.concat([s1, s2])

alice    30
bob      35
bill     32
alice    22
jo       29
dtype: int64

On remarque, cependant, que par défaut il n'y a pas de contrôle sur les labels d'indexe dupliqués. On peut corriger cela avec l'argument `verify_integrity` qui va produire une exception s'il y a des labels d'indexes qui ont un recouvrement. Évidemment, cela a un coût de calcul supplémentaire, ça n'est donc à utiliser que si c'est nécessaire.

In [16]:
try:
    pd.concat([s1, s2], verify_integrity=True)
except ValueError as e:
    print(f"erreur de concaténation:\n{e}")

erreur de concaténation:
Indexes have overlapping values: ['alice']


In [21]:
# créons deux series avec les index sans recouvrement
s1 = pd.Series(range(1000), index=[chr(x) for x in range(1000)])
s2 = pd.Series(range(1000), index=[chr(x+2000) for x in range(1000)])

# temps de concaténation avec vérification des recouvrements
%timeit pd.concat([s1, s2], verify_integrity=True)
# temps de concaténation sans vérification des recouvrements
%timeit pd.concat([s1, s2])

572 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
373 µs ± 9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Par défaut, `concat` concatène les lignes, c'est-à-dire que `s2` sera sous `s1`, mais on peut changer ce comportement un utilisant l'argument `axis`.

In [27]:
p1 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('ab'), index=list('xy'))
p2 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('ab'), index=list('zt'))

# équivalent à pd.concat([p1, p2], axis=0)
# concaténation des lignes
pd.concat([p1, p2])

,a,b
x,8,9
y,6,8
z,3,9
t,3,9


In [26]:
p1 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('ab'), index=list('xy'))
p2 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('cd'), index=list('xy'))

# concaténation des colonnes
pd.concat([p1, p2], axis=1)

,a,b,c,d
x,2,4,1,3
y,6,9,5,5


Regardons maintenant ce cas.

In [33]:
pd.concat([p1, p2])


,a,b,c
x,1.0,4,NaN
y,2.0,5,NaN
z,NaN,2,2.0
t,NaN,4,3.0


Vous remarquez que lors de la concaténation, on prend l'union des tous les labels des indexes de `p1` et `p2`, il y a donc des valeurs mises à `NaN`. On peut contrôler ce comportement de plusieurs manières que vous allons voir ci-dessous.

In [35]:
# on concatène les lignes, l'argument join décide quels labels sur l'autre 
# axe on garde (ici sur les colonnes). 

#Par défaut, join utilise la stratégie 'outer', c'est-à-dire 
# qu'on prend la concaténation des labels, si on spécifie 'inner' on prend 
# l'intersection des labels
pd.concat([p1, p2], join='inner')

,b
x,4
y,5
z,2
t,4


In [37]:
# avec join_axes, on peut spécifier les labels qu'on veut garder sous forme 
# d'un objet Index
pd.concat([p1, p2], join_axes=[p1.columns])

,a,b
x,1.0,4
y,2.0,5
z,NaN,2
t,NaN,4


In [39]:
pd.concat([p1, p2], join_axes=[pd.Index(['a', 'c'])])

,a,c
x,1.0,NaN
y,2.0,NaN
z,NaN,2.0
t,NaN,3.0


Notons que les Series et DataFrame ont une méthode `append` qui est un raccourci vers `concat`, mais avec moins d'options. 

Pour aller plus loin, voici la documentation officielle : 

http://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects

### Jointures avec `merge`

`merge` est dans l'esprit similaire au `join` en SQL. L'idée est de combiner deux DataFrame en fonction d'un critère d'égalité sur des colonnes. Regardons un exemple.

In [41]:
df1 = pd.DataFrame({'employee': ['Bob', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Sue'],
                    'hire_date': [2004, 2008, 2014]})

In [43]:
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Lisa  Engineering
2      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2      Sue       2014


On souhaite ici combiner `df1` et `df2` de manière à ce que les lignes contenant le même _employee_ soit alignées. Notre critère de merge est donc l'égalité des labels sur la colonne _employee_.

In [44]:
pd.merge(df1, df2)

,employee,group,hire_date
0,Bob,Accounting,2008
1,Lisa,Engineering,2004
2,Sue,HR,2014


Par défaut, `merge` fait un inner join (ou jointure interne) en utilisant comme critère de jointure les colonnes de même nom (ici _employee_). _inner join_ veut dire que pour joindre deux lignes il faut que le même _employee_ apparaisse dans les deux DataFrame. 

Il existe trois type de merges.

 - one-to-one, c'est celui que l'on vient de voir. C'est le merge lorqu'il n'y a pas de labels dupliqués dans les colonnes utilisées comme critère de merge.
 
 - many-to-one, c'est le merge lorsque l'une des deux colonnes contient des labels dupliqués, dans ce cas, on applique la stratégie one-to-one pour chaque lable dupliqué, donc les entrées dupliquées sont préservées.
 
 - many-to-many, c'est la stratégie lorsqu'il y a des entrées dupliquées dans les deux colonnes. Dans ce cas on fait un produit cartésien des lignes. 
 
D'une manière générale, gardez en tête que pandas fait essentiellement ce à quoi on s'attend. Regardons cela sur des exemples.
 
 
 

In [45]:
df1 = pd.DataFrame({'patient': ['Bob', 'Lisa', 'Sue'],
                    'repas': ['SS', 'SS', 'SSR']})
df2 = pd.DataFrame({'repas': ['SS', 'SSR'],
                    'explication': ['sans sel', 'sans sucre']})
print(df1)
print(df2)

  patient repas
0     Bob    SS
1    Lisa    SS
2     Sue   SSR
  explication repas
0    sans sel    SS
1  sans sucre   SSR


In [47]:
# la colonne commune pour le merge est 'repas' et dans une des colonnes 
# (sur df1), il y a des labels dupliqués, on applique la stratégie many-to-one
pd.merge(df1, df2)

,patient,repas,explication
0,Bob,SS,sans sel
1,Lisa,SS,sans sel
2,Sue,SSR,sans sucre


In [49]:
df1 = pd.DataFrame({'patient': ['Bob', 'Lisa', 'Sue'],
                    'repas': ['SS', 'SS', 'SSR']})
df2 = pd.DataFrame({'repas': ['SS', 'SS', 'SSR'],
                    'explication': ['sans sel', 'légumes', 'sans sucre']})
print(df1)
print(df2)

  patient repas
0     Bob    SS
1    Lisa    SS
2     Sue   SSR
  explication repas
0    sans sel    SS
1     légumes    SS
2  sans sucre   SSR


In [50]:
# la colonne commune pour le merge est 'repas' et dans les des colonnes 
# il y a des labels dupliqués, on applique la stratégie many-to-many
pd.merge(df1,df2)

,patient,repas,explication
0,Bob,SS,sans sel
1,Bob,SS,légumes
2,Lisa,SS,sans sel
3,Lisa,SS,légumes
4,Sue,SSR,sans sucre


Dans un merge, on peut contrôler les colonnes à utiliser comme critère de merge. Regardons ces différents cas sur des exemples.

In [53]:
df1 = pd.DataFrame({'employee': ['Bob', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Sue'],
                    'hire_date': [2004, 2008, 2014]})
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Lisa  Engineering
2      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2      Sue       2014


In [54]:
# on décide d'utiliser la colonne 'employee' comme critère de merge
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Lisa,Engineering,2004
2,Sue,HR,2014


In [55]:
df1 = pd.DataFrame({'employee': ['Bob', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'HR']})
df2 = pd.DataFrame({'name': ['Lisa', 'Bob', 'Sue'],
                    'hire_date': [2004, 2008, 2014]})
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Lisa  Engineering
2      Sue           HR
   hire_date  name
0       2004  Lisa
1       2008   Bob
2       2014   Sue


In [57]:
# mais on peut également définir un nom de colonne différent 
# à gauche et à droite
m = pd.merge(df1,df2, left_on='employee', right_on='name')
print(m)

  employee        group  hire_date  name
0      Bob   Accounting       2008   Bob
1     Lisa  Engineering       2004  Lisa
2      Sue           HR       2014   Sue


In [58]:
# dans ce cas, comme on garde les colonnes utilisées comme critère dans 
# le résultat du merge, on peut effacer la colonne inutile ainsi
m.drop('name', axis=1)


,employee,group,hire_date
0,Bob,Accounting,2008
1,Lisa,Engineering,2004
2,Sue,HR,2014


`merge` permet également de contrôler la stratégie à appliquer lorsqu'il y a des valeurs dans une colonne utilisée comme critère merge absente dans l'autre colonne. C'est ce que l'on appelle jointure à gauche, jointure à droite, jointure interne (comportement par défaut) et jointure externe. Pour ceux qui ne sont pas familié avec ces notions, regardons des exemples. 

In [59]:
df1 = pd.DataFrame({'name': ['Bob', 'Lisa', 'Sue'],
                    'pulse': [70, 63, 81]})
df2 = pd.DataFrame({'name': ['Eric', 'Bob', 'Marc'],
                    'weight': [60, 100, 70]})
print(df1)
print(df2)

   name  pulse
0   Bob     70
1  Lisa     63
2   Sue     81
   name  weight
0  Eric      60
1   Bob     100
2  Marc      70


In [61]:
# la colonne 'name' est critère de merge dans les deux DataFrame.
# Seul Bob existe dans les deux colonnes. Dans un inner join 
# (le cas par défaut) on ne garde que les lignes pour lesquels il y a une 
# même valeur présente à gauche et à droite
pd.merge(df1, df2) # équivalent à pd.merge(df1, df2, how='inner')

,name,pulse,weight
0,Bob,70,100


In [63]:
# le outer join va au contraire faire une union des lignes et compléter ce 
# qui manque avec NaN
pd.merge(df1, df2, how='outer')

,name,pulse,weight
0,Bob,70.0,100.0
1,Lisa,63.0,NaN
2,Sue,81.0,NaN
3,Eric,NaN,60.0
4,Marc,NaN,70.0


In [64]:
# le left join ne garde que les valeurs de la colonne de gauche 
pd.merge(df1, df2, how='left')

,name,pulse,weight
0,Bob,70,100.0
1,Lisa,63,NaN
2,Sue,81,NaN


In [65]:
# et le right join ne garde que les valeurs de la colonne de droite 
pd.merge(df1, df2, how='right')

,name,pulse,weight
0,Bob,70.0,100
1,Eric,NaN,60
2,Marc,NaN,70


Pour aller plus loin, vous pouvez lire la documentation. Vous verrez notamment que vous pouvez merger sur les indexes (au lieu des colonnes) ou le cas ou vous avez des colonnes de même noms qui ne font pas partie du critère de merge&nbsp;:

http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

### Regroupement avec `groupby`

Regardons maintenant cette notion de groupement. Il s'agit d'une notion très puissante avec de nombreuses options que nous ne couvrirons que partiellement. 
La logique derrière `groupby` est de créer des groupes dans une DataFrame en fonction des valeurs d'une (ou plusieurs) colonne(s), toutes les lignes contenant la même valeur sont dans le même groupe. On peut ensuite appliquer à chaque groupe des opérations qui sont :

 - soit des calculs sur chaque groupe ;
 - soit un filtre sur chaque groupe qui peut garder ou supprimer un groupe ;
 - soit une transformation qui va modifier tout le groupe (par exemple, pour centrer les valeurs sur la moyenne du groupe).
 
Regardons quelques exemples.

In [66]:
d = pd.DataFrame({'key': list('ABCABC'), 'val': range(6)})
print(d)

  key  val
0   A    0
1   B    1
2   C    2
3   A    3
4   B    4
5   C    5


In [72]:
# utilisons comme colonne de groupement 'key'
g = d.groupby('key')
print(g)


`groupby` produit un nouvel objet, mais ne fait aucun calcul. Les calculs seront affectués lors de l'appel de fonction sur ce nouvel objet. Par exemple, calculons la somme pour chaque groupe.

In [73]:
g.sum()

,val
key,
A,3
B,5
C,7


`groupby` peut utiliser comme critère de groupement une colonne, une liste de colonne, ou un index (c'est notamment utile pour les Series). 

On particularité de `groupby` est que le critère de groupement devient un index dans le nouvel objet généré. L'avantage est que l'on a maintenant un accès optimisé sur ce critère, mais l'inconvénient est que sur certaines opérations qui détruise l'index on peut perdre ce critère. On peut contrôler ce comportement avec `as_index`.

In [74]:
g = d.groupby('key', as_index=False)
g.sum()

,key,val
0,A,3
1,B,5
2,C,7
